In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk # For StopWords
from nltk.corpus import stopwords # For Stopwords
from nltk.stem.porter import PorterStemmer # For stemming words
from sklearn.feature_extraction.text import CountVectorizer # For Vectorizing words
from sklearn.model_selection import train_test_split # For test train split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data = pd.read_csv("Dataset/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
data.shape

(50000, 2)

In [5]:
data = data.sample(50000).reset_index()
data.shape

(50000, 3)

In [6]:
data['review'][0]

"I saw this movie on VHS some time ago (27 Jan 2003), just because of the name of Paul Rudd on the cover. I liked his performance in `The Object of My Affection' very much and I really expected a good work. However, I found this film a complete mess. The story has a very confused screenplay and the characters are not well developed. Further, the low-budget special effects do not help much. I do not know the previous generation of Gen-Y Cops, but this next generation is not good. I do not recall exactly why I gave this grade (and I do not intent to see this movie again), but my vote is four.<br /><br />Title (Brazil) : `Gen-Y Cops A Nova Geração' (`Gen-Y Cops The Next Generation')"

In [7]:
data['sentiment'].replace({'positive': 1, 'negative': 0}, inplace=True)
data.head()

,index,review,sentiment
0,12482,I saw this movie on VHS some time ago (27 Jan ...,0
1,9923,Since I am required to write minimum of 10 lin...,0
2,12991,I was one of many that expected to see a glori...,1
3,23199,"American icon Henry Fonda portrays ""Elegant"" J...",1
4,38069,This is one very confusing movie. The film is ...,0


In [8]:
data.iloc[2].review

"I was one of many that expected to see a glorified, Yankee-doodle dandy portrayal of a day that (as famously quoted) should live in infamy, rather than glory. How wrong I was. These guys were there, right in the middle of it, and the pictures they returned are both amazing and heartbreaking. And yet it all occurred on a chance trip to the world trade centre on September 11, 2001.<br /><br />Two French filmmakers were compiling a documentary about life as a NY firefighter, particularly from the perspective of a young rookie coming up through the ranks. At the beginning we see much of this footage, just to remind us that there was no thought to producing a film about terrorism. This was intended to be a film about regular people earning an honest living helping others, and the beauty of the film is that it never loses this edge.<br /><br />While investigating a suspect gas line (I think, my memory's a little hazy on that), we suddenly hear a plane fly overhead. The camera pans up to rev

In [9]:
# Removing all HTML Tags
clean = re.compile('<.*?>')
re.sub(clean, '', data.iloc[2]. review)

"I was one of many that expected to see a glorified, Yankee-doodle dandy portrayal of a day that (as famously quoted) should live in infamy, rather than glory. How wrong I was. These guys were there, right in the middle of it, and the pictures they returned are both amazing and heartbreaking. And yet it all occurred on a chance trip to the world trade centre on September 11, 2001.Two French filmmakers were compiling a documentary about life as a NY firefighter, particularly from the perspective of a young rookie coming up through the ranks. At the beginning we see much of this footage, just to remind us that there was no thought to producing a film about terrorism. This was intended to be a film about regular people earning an honest living helping others, and the beauty of the film is that it never loses this edge.While investigating a suspect gas line (I think, my memory's a little hazy on that), we suddenly hear a plane fly overhead. The camera pans up to reveal a commercial jet tor

In [10]:
# Function to clean HTML Tags:
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [11]:
data['review'] = data['review'].apply(clean_html)

In [12]:
data.iloc[2].review

"I was one of many that expected to see a glorified, Yankee-doodle dandy portrayal of a day that (as famously quoted) should live in infamy, rather than glory. How wrong I was. These guys were there, right in the middle of it, and the pictures they returned are both amazing and heartbreaking. And yet it all occurred on a chance trip to the world trade centre on September 11, 2001.Two French filmmakers were compiling a documentary about life as a NY firefighter, particularly from the perspective of a young rookie coming up through the ranks. At the beginning we see much of this footage, just to remind us that there was no thought to producing a film about terrorism. This was intended to be a film about regular people earning an honest living helping others, and the beauty of the film is that it never loses this edge.While investigating a suspect gas line (I think, my memory's a little hazy on that), we suddenly hear a plane fly overhead. The camera pans up to reveal a commercial jet tor

In [13]:
def convert_lower(text):
    return text.lower()

In [14]:
data['review'] = data['review'].apply(convert_lower)

In [15]:
data.head()

,index,review,sentiment
0,12482,i saw this movie on vhs some time ago (27 jan ...,0
1,9923,since i am required to write minimum of 10 lin...,0
2,12991,i was one of many that expected to see a glori...,1
3,23199,"american icon henry fonda portrays ""elegant"" j...",1
4,38069,this is one very confusing movie. the film is ...,0


In [16]:
def remove_special(text):
    x = ""
    
    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + " "
    return x

In [17]:
data.iloc[2].review

"i was one of many that expected to see a glorified, yankee-doodle dandy portrayal of a day that (as famously quoted) should live in infamy, rather than glory. how wrong i was. these guys were there, right in the middle of it, and the pictures they returned are both amazing and heartbreaking. and yet it all occurred on a chance trip to the world trade centre on september 11, 2001.two french filmmakers were compiling a documentary about life as a ny firefighter, particularly from the perspective of a young rookie coming up through the ranks. at the beginning we see much of this footage, just to remind us that there was no thought to producing a film about terrorism. this was intended to be a film about regular people earning an honest living helping others, and the beauty of the film is that it never loses this edge.while investigating a suspect gas line (i think, my memory's a little hazy on that), we suddenly hear a plane fly overhead. the camera pans up to reveal a commercial jet tor

In [18]:
remove_special(data.iloc[2].review)

'i was one of many that expected to see a glorified  yankee doodle dandy portrayal of a day that  as famously quoted  should live in infamy  rather than glory  how wrong i was  these guys were there  right in the middle of it  and the pictures they returned are both amazing and heartbreaking  and yet it all occurred on a chance trip to the world trade centre on september 11  2001 two french filmmakers were compiling a documentary about life as a ny firefighter  particularly from the perspective of a young rookie coming up through the ranks  at the beginning we see much of this footage  just to remind us that there was no thought to producing a film about terrorism  this was intended to be a film about regular people earning an honest living helping others  and the beauty of the film is that it never loses this edge while investigating a suspect gas line  i think  my memory s a little hazy on that   we suddenly hear a plane fly overhead  the camera pans up to reveal a commercial jet tor

In [19]:
data['review'] = data['review'].apply(remove_special)
data.head()

,index,review,sentiment
0,12482,i saw this movie on vhs some time ago 27 jan ...,0
1,9923,since i am required to write minimum of 10 lin...,0
2,12991,i was one of many that expected to see a glori...,1
3,23199,american icon henry fonda portrays elegant j...,1
4,38069,this is one very confusing movie the film is ...,0


In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ritwikghosh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [22]:
def remove_stopwords(text): 
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [23]:
data.iloc[2].review

'i was one of many that expected to see a glorified  yankee doodle dandy portrayal of a day that  as famously quoted  should live in infamy  rather than glory  how wrong i was  these guys were there  right in the middle of it  and the pictures they returned are both amazing and heartbreaking  and yet it all occurred on a chance trip to the world trade centre on september 11  2001 two french filmmakers were compiling a documentary about life as a ny firefighter  particularly from the perspective of a young rookie coming up through the ranks  at the beginning we see much of this footage  just to remind us that there was no thought to producing a film about terrorism  this was intended to be a film about regular people earning an honest living helping others  and the beauty of the film is that it never loses this edge while investigating a suspect gas line  i think  my memory s a little hazy on that   we suddenly hear a plane fly overhead  the camera pans up to reveal a commercial jet tor

In [24]:
remove_stopwords(data.iloc[2].review)

['one',
 'many',
 'expected',
 'see',
 'glorified',
 'yankee',
 'doodle',
 'dandy',
 'portrayal',
 'day',
 'famously',
 'quoted',
 'live',
 'infamy',
 'rather',
 'glory',
 'wrong',
 'guys',
 'right',
 'middle',
 'pictures',
 'returned',
 'amazing',
 'heartbreaking',
 'yet',
 'occurred',
 'chance',
 'trip',
 'world',
 'trade',
 'centre',
 'september',
 '11',
 '2001',
 'two',
 'french',
 'filmmakers',
 'compiling',
 'documentary',
 'life',
 'ny',
 'firefighter',
 'particularly',
 'perspective',
 'young',
 'rookie',
 'coming',
 'ranks',
 'beginning',
 'see',
 'much',
 'footage',
 'remind',
 'us',
 'thought',
 'producing',
 'film',
 'terrorism',
 'intended',
 'film',
 'regular',
 'people',
 'earning',
 'honest',
 'living',
 'helping',
 'others',
 'beauty',
 'film',
 'never',
 'loses',
 'edge',
 'investigating',
 'suspect',
 'gas',
 'line',
 'think',
 'memory',
 'little',
 'hazy',
 'suddenly',
 'hear',
 'plane',
 'fly',
 'overhead',
 'camera',
 'pans',
 'reveal',
 'commercial',
 'jet',
 'to

In [25]:
data['review'] = data['review'].apply(remove_stopwords)

In [26]:
ps = PorterStemmer()

In [27]:
y = []
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [28]:
data['review'] = data['review'].apply(stem_words)
data.head()

,index,review,sentiment
0,12482,"[saw, movi, vh, time, ago, 27, jan, 2003, name...",0
1,9923,"[sinc, requir, write, minimum, 10, line, garba...",0
2,12991,"[one, mani, expect, see, glorifi, yanke, doodl...",1
3,23199,"[american, icon, henri, fonda, portray, eleg, ...",1
4,38069,"[one, confus, movi, film, hard, follow, plot, ...",0


In [29]:
# Join Back
def join_back(list_input):
    return " ".join(list_input)

In [30]:
data['review'] = data['review'].apply(join_back)
data.head()

,index,review,sentiment
0,12482,saw movi vh time ago 27 jan 2003 name paul rud...,0
1,9923,sinc requir write minimum 10 line garbag deser...,0
2,12991,one mani expect see glorifi yanke doodl dandi ...,1
3,23199,american icon henri fonda portray eleg john ho...,1
4,38069,one confus movi film hard follow plot seem mak...,0


In [31]:
max_f = -1

In [108]:
cv= CountVectorizer(max_features=3700, ngram_range= (1,2))

x = cv.fit_transform(data['review']).toarray()
y = data.iloc[:,-1].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
    
# clf1 = GaussianNB()
# clf2 = MultinomialNB()
clf3 = BernoulliNB()

# clf1.fit(x_train, y_train)
# clf2.fit(x_train, y_train)
clf3.fit(x_train, y_train)

# y_pred1 = clf1.predict(x_test)
# y_pred2 = clf2.predict(x_test)
y_pred3 = clf3.predict(x_test)

# a_score1 = accuracy_score(y_test, y_pred1)
# a_score2 = accuracy_score(y_test, y_pred2)
a_score3 = accuracy_score(y_test, y_pred3)

print(a_score3)
# if(a_score1 > max_f):
#     print(i)
#     print("Gaussian:", a_score1)
#     max_f = a_score1
#     print()

# if(a_score2 > max_f): is 
#     print(i)
#     print("Multinomial:", a_score2)
#     max_f = a_score2
#     print()

# if(a_score3 > max_f):
#     print(i)
#     print("Bernaulli:", a_score3)
#     max_f = a_score3
#     print()

0.8552


In [34]:
# for i in range(9500, 10000, 100):
    
#     cv= CountVectorizer(max_features=i, ngram_range= (1,2))
    
#     x = cv.fit_transform(data['review']).toarray()
#     y = data.iloc[:,-1].values

#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
        
#     clf1 = GaussianNB()
#     clf2 = MultinomialNB()
#     clf3 = BernoulliNB()

#     clf1.fit(x_train, y_train)
#     clf2.fit(x_train, y_train)
#     clf3.fit(x_train, y_train)

#     y_pred1 = clf1.predict(x_test)
#     y_pred2 = clf2.predict(x_test)
#     y_pred3 = clf3.predict(x_test)

#     a_score1 = accuracy_score(y_test, y_pred1)
#     a_score2 = accuracy_score(y_test, y_pred2)
#     a_score3 = accuracy_score(y_test, y_pred3)

#     if(a_score1 > max_f):
#         print(i)
#         print("Gaussian:", a_score1)
#         max_f = a_score1
#         print()

#     if(a_score2 > max_f):
#         print(i)
#         print("Multinomial:", a_score2)
#         max_f = a_score2
#         print()

#     if(a_score3 > max_f):
#         print(i)
#         print("Bernaulli:", a_score3)
#         max_f = a_score3
#         print()

In [109]:
pickle.dump(clf3, open('mymodels/model.pkl', 'wb'))

In [110]:
pickle.dump(cv, open('mymodels/bow.pkl', 'wb'))

In [111]:
y_test

array([0, 0, 1, ..., 1, 1, 0])

In [112]:
import pickle

In [113]:
words = pickle.load(open("mymodels/bow.pkl", 'rb'))

In [114]:
words

CountVectorizer(max_features=3700, ngram_range=(1, 2))

In [115]:
model = pickle.load(open("mymodels/model.pkl", 'rb'))

In [116]:
model

BernoulliNB()

In [117]:
words.transform(["no"])

<1x3700 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [122]:
model.predict(words.transform([""]))[0]

1

In [123]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 458 kB/s eta 0:00:01
     |████████████████████████████████| 61 kB 318 kB/s eta 0:00:01
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
     |████████████████████████████████| 155 kB 714 kB/s eta 0:00:01
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
     |████████████████████████████████| 136 kB 689 kB/s eta 0:00:01
You should consider upgrading via the '/Users/ritwikghosh/Desktop/codes/Python/Sentiment-Analysis-Of-Movie-Review/venv/bin/python -m pip install --upgrade pip' command.


In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
limit=-0.3

In [13]:
def set_score(lim):
    limit=lim

In [2]:
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
    
    result={}
      
    print("Overall sentiment dictionary is : ", sentiment_dict) 
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
    
#     result{'neg_rating':"Overall sentiment dictionary is : "+ str(sentiment_dict)}
    result['neg_rating']=str('{0:.3g}'.format(sentiment_dict['neg']*100))
    result['neu_rating']=str('{0:.3g}'.format(sentiment_dict['neu']*100))
    result['pos_rating']=str('{0:.3g}'.format(sentiment_dict['pos']*100))
    result['comp_rating']=str('{0:.3g}'.format(sentiment_dict['compound']*100))
  
    print("Sentence Overall Rated As", end = " ")
    
#     result+="Sentence Overall Rated As "
  
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        print("Positive")
        result['res']="Positive"
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        print("Negative")
        result['res']="Negative"
  
    else : 
        print("Neutral")
        result['res']="Neutral"
    
    if sentiment_dict['compound']< limit.get()/100:
        print('The sentence is not recommended to be posted')
        result['verd']="The sentence is not recommended to be posted"
        
    else:
        print('The sentence is recommended to be posted')
        result['verd']="The sentence is recommended to be posted"
    return result

In [34]:
sentiment_scores("if you want to have a good day, try our product!") 

Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.5411}
sentence was rated as  0.0 % Negative
sentence was rated as  66.7 % Neutral
sentence was rated as  33.300000000000004 % Positive
Sentence Overall Rated As Positive
The sentence is recommended to be posted


{'neg_rating': '0',
 'neu_rating': '66.7',
 'pos_rating': '33.3',
 'comp_rating': '54.1',
 'res': 'Positive',
 'verd': 'The sentence is recommended to be posted'}

In [3]:
from tkinter import *
from tkinter import messagebox
import json 
from tkinter.filedialog import askopenfilename

In [4]:
file = open('credentials.json')
creds = json.load(file)
file.close()

In [31]:
window=Tk()
window.title("Sentiment Analyser Twitter Tool")
window.resizable(False,False)

content=StringVar()
# entry=Entry(window, textvariable=content)
# entry.grid(row=0, column=0, rowspan=2)

label=Label(window, text="Enter text here:", height=2,font=("Arial Bold", 27))
label.grid(row=0, column=0, columnspan=2)
#text area
text=Text(window, borderwidth = 3, relief=RAISED, wrap=WORD)
text.grid(row=1, column=0, columnspan=2, rowspan=15, padx=20)

#analyse button
b1=Button(window, text="Analyse", command=analyse)
b1.grid(row=20, column=0, padx=20, pady=10)

#posting on twitter button
b2=Button(window, text="Post on Twitter", command=post)
b2.grid(row=20, column=1)

#add media button
add_media=Button(window, text="Add media", command=fetch_media)
add_media.grid(row=20, column=4)

#remove media button
rm_media=Button(window, text="Remove media", command=remove_media)
rm_media.grid(row=20, column=5)

label=Label(window, text="Analysis results:", height=2,font=("Arial Bold", 27))
label.grid(row=0, column=3, columnspan=3)
#positive colour and score labels
pos_val=StringVar()
label=Label(window, height=1, width=2, relief=RAISED,bg='green')
label.grid(row=2, column=3, padx=15)
label=Label(window, text="Positive:", height=1)
label.grid(row=2, column=4)
pos_label=Label(window, textvariable=pos_val, height=1, width=5, relief=RAISED)
# pos_text=Text(window, height=1, width=10, wrap="word",relief=RAISED)
# pos_text.grid(row=0, column=4)
pos_label.grid(row=2, column=5)

#neutral colour and score labels
neu_val=StringVar()
label=Label(window, height=1, width=2, relief=RAISED,bg='blue')
label.grid(row=3, column=3)
label=Label(window, text="Neutral:", height=1)
label.grid(row=3, column=4)
neu_label=Label(window, textvariable=neu_val, height=1, width=5, relief=RAISED)
# neu_text=Text(window, height=1, width=10, wrap="word")
# neu_text.grid(row=5, column=4)
neu_label.grid(row=3, column=5)

#negative colour and score labels
neg_val=StringVar()
label=Label(window, height=1, width=2, relief=RAISED,bg='red')
label.grid(row=4, column=3)
label=Label(window, text="Negative:", height=1)
label.grid(row=4, column=4)
neg_label=Label(window, textvariable=neg_val, height=1, width=5, relief=RAISED)
# neg_text=Text(window, height=1, width=10, wrap="word")
# neg_text.grid(row=10, column=4)
neg_label.grid(row=4, column=5)

#compound score label
col=StringVar()
if col.get()==None or col.get()=="":
    col.set("#ffffff")
comp_val=StringVar()
# col_label1=Label(window, height=1, width=2, relief=RAISED,bg=col.get())
# col_label1.grid(row=3, column=3)
label=Label(window, text="Compound:", height=1)
label.grid(row=5, column=4)
comp_label=Label(window, textvariable=comp_val, height=1, width=5, relief=RAISED)
# neg_text=Text(window, height=1, width=10, wrap="word")
# neg_text.grid(row=10, column=4)
comp_label.grid(row=5, column=5)

#Sentiment visualisation colour
label=Label(window, text="Sentiment\nvisualisation:", height=2)
label.grid(row=7, column=4)
# print(col.get())
col_label=Label(window, height=2, width=5, relief=RAISED,bg=col.get())
col_label.grid(row=7, column=5)

#result label
res_val=StringVar()
res_label=Label(window, textvariable=res_val, height=2, width=35)
res_label.grid(row=9, column=4, columnspan=2)

#limit slider
limit=DoubleVar()
limit.set(0)
label=Label(window, text="Set sentiment\nscore limit:", height=2)
label.grid(row=11, column=4,)
lim_scale=Scale(window, variable = limit, from_ = -100, to = 100, orient = HORIZONTAL, sliderlength=10)
lim_scale.grid(row=11, column=5)

#verdict label
verd_val=StringVar()
verd_label=Label(window, textvariable=verd_val, height=2, width=40)
verd_label.grid(row=13, column=4, columnspan=2)


hasMedia = StringVar()
hasMedia.set("False")
media_path=StringVar()
media_path.set("")
#media path label
media_val=StringVar()
media_label=Label(window, textvariable=media_val, height=2, width=40, wraplength = 300)
media_label.grid(row=14, column=4, columnspan=2)

window.mainloop()

In [5]:
def remove_media():
    media_val.set("")
    hasMedia.set("False")
    media_path.set("")

In [30]:
def fetch_media():
    formats=['jpg','jpeg','png','gif','webp']
    media_path.set(askopenfilename(filetypes=[("Image files","*.jpg"),("Image files","*.gif"),("Image files","*.jpeg"),("Image files","*.png")]))
    filename=media_path.get().split('/')[-1]
    if not filename.isspace() and len(filename) != 0:
        if filename.split(".")[-1].lower() in formats:
            media_val.set("Media: "+filename)
            hasMedia.set("True")
        else:
            messagebox.showwarning("Warning", "Unsupported media format! Must be JPG, PNG, GIF or WEBP", parent=window)
            media_path.set("")
            media_val.set("")
            hasMedia.set("False")
    else:
        media_path.set("")
        media_val.set("")
        hasMedia.set("False")

In [7]:
def analyse():
#     pos_text.configure(state="normal")
#     pos_text.delete(1.0, END)
    
#     neg_text.configure(state="normal"
#     neg_text.delete(1.0, END)
    
#     neu_text.configure(state="normal")
#     neu_text.delete(1.0, END)
    
    sentence=text.get("1.0",END)
    res=(sentiment_scores(sentence))
    print(res)
    
    if not sentence.isspace() and len(sentence) != 0:
    
        pos_val.set(res['pos_rating']+"%")

        neu_val.set(res['neu_rating']+"%")

        neg_val.set(res['neg_rating']+"%")

        comp_val.set(res['comp_rating']+"%")

        res_val.set("The text was overall rated as "+res['res']+".")
        
        verd_val.set(res['verd'])

        hexcode= '#%02x%02x%02x' % (int(float(res['neg_rating'])/100*255),int(float(res['pos_rating'])/100*255),int(float(res['neu_rating'])/100*255))
    #     hexcode="#"+str(hex(255-int(int(res['neg_rating'])/100*255)).split('x')[-1])+str(hex(255-int(int(res['pos_rating'])/100*255)).split('x')[-1])+"ff"
    #     col.set(hexcode)
    #     print(col.get())
        if hexcode == "#000000":
            hexcode = "#ffffff"
        col_label.configure(bg=hexcode)
        print(hexcode)
    
    else:
        pos_val.set("")

        neu_val.set("")

        neg_val.set("")

        comp_val.set("")

        res_val.set("")
        
        verd_val.set("")
        
        col_label.configure(bg='#ffffff')
    
#     pos_text.insert(END, res['pos_rating'])
#     pos_text.configure(state="disabled")
    
#     neg_text.insert(END, res['neg_rating'])
#     neg_text.configure(state="disabled")
    
#     neu_text.insert(END, res['neu_rating'])
#     neu_text.configure(state="disabled")

In [8]:
def post():
    sentence=text.get("1.0",END)
    if len(sentence) > 280:
        messagebox.showwarning("Warning", "Text must be up to 280 characters to post on Twitter!", parent=window)
    elif sentence.isspace() or len(sentence) == 0:
        messagebox.showwarning("Warning", "No message to post!", parent=window)
    else:
        choice=messagebox.askquestion("Asking confirmation", "Are you sure you want to post this on Twitter?", parent=window)
        if choice == "yes":
            try:
                print("length of message=", sentence)
                consumer_key = creds["consumer_key"]
                consumer_secret =creds["consumer_secret"]
                access_token =creds["access_token"]
                access_token_secret =creds["access_token_secret"]
                # authentication of consumer key and secret 
                auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
                # Authentication of access token and secret 
                auth.set_access_token(access_token, access_token_secret) 
                api = tweepy.API(auth)
                print(hasMedia.get())
                if hasMedia.get()=="True":
                    print('with media\n')
                    print(media_path)
                    api.update_with_media(media_path.get(), sentence)
        #                 api.update_status(status = sentence) 

                else:
                    api.update_status(status=sentence)

                text.delete(1.0, END)
                pos_val.set("")
                neu_val.set("")
                neg_val.set("")
                comp_val.set("")
                res_val.set("")
                col_label.configure(bg="#ffffff")
                verd_val.set("")
                media_val.set("")
                hasMedia.set("False")
                media_path.set("")

            except:
                messagebox.showerror("showerror", "Something went wrong!", parent=window)
            else:
                messagebox.showinfo("Info", "Successfully posted on Twitter!", parent=window)

In [11]:
!pip install tweepy

  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
You should consider upgrading via the '/Users/ritwikghosh/Desktop/codes/Python/Sentiment-Analysis-Of-Movie-Review/venv/bin/python -m pip install --upgrade pip' command.


In [9]:
import tweepy

In [2]:
import tweepy
# personal details 
my_consumer_key ="UUuvP0B5Cofa4tbDLSnIiARL5"
my_consumer_secret ="mETZ4ZIOAUjyWiV0T7MPUQyLcOB01e5Q4qyX9fNdaNAmPRLR3o"
my_access_token ="860092505525673984-qkLiWt6uEZB0lSr9IBe9gNBtL0UNfwU"
my_access_token_secret ="pVcatZMDUBSkDQJQYsxO1sevD6ywzDk3w115brVPwmIcP"
# authentication of consumer key and secret 
my_auth = tweepy.OAuthHandler(my_consumer_key, my_consumer_secret) 
# Authentication of access token and secret 
my_auth.set_access_token(my_access_token, my_access_token_secret) 
my_api = tweepy.API(my_auth)
my_api.update_status(status=" Tweet using python. Hello world!")

Status(_api=<tweepy.api.API object at 0x1092d0910>, _json={'created_at': 'Thu Dec 10 11:00:07 +0000 2020', 'id': 1336989062956138496, 'id_str': '1336989062956138496', 'text': 'Tweet using python. Hello world!', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 860092505525673984, 'id_str': '860092505525673984', 'name': 'Ritwik Ghosh', 'screen_name': 'ritwikghosh1010', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 3, 'friends_count': 2, 'listed_count': 0, 'created_at': 'Thu May 04 11:23:27 +0000 2017', 'favourites_count': 9, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 12, 'lang': None, 'contributors_enabled': False, '

In [32]:
len("ectetur libero. Consectetur adipiscing elit pellentesque habitant morbi. Est lorem ipsum dolor sit. Tincidunt eget nullam non nisi est sit amet facilisis. Ornare aenean la facilisi etiam dignissim. Eu mi bibendum lorem dolor sed viverra ipsum nunc aliquet bibendum. Sit amet volut")

280

In [70]:
hexcode='#%02x%02x%02x' % (int(255-90/100*255),int(255-50.3/100*255),int(255-10.3/100*255))
print(hexcode)

#197ee4


In [34]:
import ctypes  # An included library with Python install.
def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)
Mbox('Your title', 'Your text', 1)

AttributeError: module 'ctypes' has no attribute 'windll'